# MAKE SURE TO DELETE YOUR ENDPOINT

To get this working you will need to install git lfs, open a new terminal and follow the instructions below line by line

```
conda install -c conda-forge git-lfs
git lfs install
```

```
GIT_SSL_NO_VERIFY=1 git lfs clone https://huggingface.co/sshleifer/distilbart-cnn-12-6
```

cd into the created directory and run

```
tar zcvf model.tar.gz *
```

then copy your model.tar.gz file to the home directory

# MAKE SURE TO DELETE YOUR ENDPOINT

In [38]:
import boto3
import os
import sagemaker

bucket = sagemaker.session.Session().default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [40]:
#from sagemaker.inputs import TrainingInput
input_prefix = "HuggingFace/input"

input_paths = {}
output_path = os.path.join("s3://", bucket, output_prefix)

data_name="model.tar.gz"
pre_key = os.path.join(input_prefix, "test")
data_path = os.path.join("s3://", bucket, pre_key, fname)
s3_client.upload_file(fname, bucket, os.path.join(pre_key, fname))
input_paths[data_name] = TrainingInput(
        data_path, distribution="ShardedByS3Key", content_type="application/jsonlines"
)

#print("Uploaded {} data to {} and defined input path".format(data_name, data_path))

In [41]:
# This just checks the file has been correctly uploaded and is the right size
# fnametest = fname + "test"
#s3_client.download_file(bucket, os.path.join(pre_key, fname),fnametest)

In [42]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface.model import HuggingFaceModel
sess = sagemaker.Session()

role = get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   role=role,
   model_data= data_path,       # path to your model and script                # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# MAKE SURE TO DELETE YOUR ENDPOINT

In [43]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [44]:
# example request: you always need to define "inputs"
data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

# request
predictor.predict(data)

[{'generated_text': ' SiPix Digital Camera! Call 09061221066 fromm landline. Delivery within 28 days. Camera - You are awarded a SiPIX Digital Camera. call 0906121066. Call 0800 555 555 111 or visit www.siPix.com/SiPix .'}]

In [45]:
# request
predictor.predict({
	"inputs": "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.",
})

[{'generated_text': ' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It was the first structure to reach a height of 300 metres . It is now taller than the Chrysler Building in New York City by 5.2 metres (17 ft) Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France .'}]

In [39]:
predictor.delete_endpoint()